In [1]:
pip install git+https://github.com/Desklop/Uk_Stemmer

  Cloning https://github.com/Desklop/Uk_Stemmer to /tmp/pip-req-build-6rbopptj
  Running command git clone -q https://github.com/Desklop/Uk_Stemmer /tmp/pip-req-build-6rbopptj


In [2]:
pip install spark-nlp==3.4.0

  Using cached spark_nlp-3.4.0-py2.py3-none-any.whl (140 kB)
  Attempting uninstall: spark-nlp
    Found existing installation: spark-nlp 3.3.4
    Uninstalling spark-nlp-3.3.4:
      Successfully uninstalled spark-nlp-3.3.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nlu 3.3.1 requires spark-nlp<3.4.0,>=3.3.0, but you have spark-nlp 3.4.0 which is incompatible.


In [3]:
pip install pyspark==3.0.2

In [4]:
pip install nlu pyspark==3.0.2

  Using cached spark_nlp-3.3.4-py2.py3-none-any.whl (133 kB)
  Attempting uninstall: spark-nlp
    Found existing installation: spark-nlp 3.4.0
    Uninstalling spark-nlp-3.4.0:
      Successfully uninstalled spark-nlp-3.4.0


In [5]:
!pip install pyenchant

In [6]:
!apt-get install libenchant1c2a

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libenchant1c2a is already the newest version (1.6.0-11.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from uk_stemmer import UkStemmer

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp

In [10]:
import nlu

In [11]:
import enchant

In [12]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
pd.options.display.float_format = "{:,.2f}".format
np.set_printoptions(suppress=True)
nltk.download('stopwords')
nltk.download('punkt')
random_state = 8080

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
ls

drive/  sample_data/


In [15]:
df = pd.read_csv("/content/drive/MyDrive/full_data.csv")

In [16]:
df.head()

,crm_customer_id,created,NAME_LOWEST
0,50081964,2021-01-25 14:21:07.610,Петлі меблеві
1,50081964,2021-01-25 14:21:07.610,Петлі меблеві
2,50081964,2021-01-29 13:53:52.920,Лотки для столових приборів
3,50081964,2021-05-31 12:21:22.080,Інсектициди
4,50081964,2021-02-20 15:02:06.730,Рейлінги для кухні


In [17]:
df.columns = ["customer_id", "date", "category"]

In [18]:
df.head(2)

,customer_id,date,category
0,50081964,2021-01-25 14:21:07.610,Петлі меблеві
1,50081964,2021-01-25 14:21:07.610,Петлі меблеві


In [19]:
df.shape

(1412105, 3)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1412105 entries, 0 to 1412104
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   customer_id  1412105 non-null  int64 
 1   date         1412105 non-null  object
 2   category     1412105 non-null  object
dtypes: int64(1), object(2)
memory usage: 32.3+ MB


In [21]:
print(f"number of categories: {df.category.nunique()}")

number of categories: 2129


In [22]:
print(f"number of customers: {df.customer_id.nunique()}")

number of customers: 20114


# NLP feature engineering

1. make all words lowercase
2. remove stopwords
3. preform stemming

In [23]:
df.head()

,customer_id,date,category
0,50081964,2021-01-25 14:21:07.610,Петлі меблеві
1,50081964,2021-01-25 14:21:07.610,Петлі меблеві
2,50081964,2021-01-29 13:53:52.920,Лотки для столових приборів
3,50081964,2021-05-31 12:21:22.080,Інсектициди
4,50081964,2021-02-20 15:02:06.730,Рейлінги для кухні


In [24]:
df.category.unique()[:20]

array(['Петлі меблеві', 'Лотки для столових приборів', 'Інсектициди',
       'Рейлінги для кухні', 'Алюмінієвий профіль', 'Стопори дверні',
       'Зубна паста', 'Куточок алюмінієвий', 'Мішки для сміття',
       'Господарчі рукавички', 'Серветки для прибирання',
       'Аератори для крана', 'Засоби для ванних кімнат', 'Швабри',
       'Засоби для кухні', 'Засоби для догляду за унітазом',
       'Клей побутовий', 'Губки господарчі', 'Пральний порошок',
       'Клей для шпалер'], dtype=object)

In [25]:
ndf = df.drop_duplicates(subset=["customer_id", "category"])

In [26]:
print(f"distinct records: {round((ndf.shape[0] * 100) / df.shape[0])}%")

distinct records: 38%


In [27]:
ndf["category"] = ndf["category"].str.lower()

In [28]:
ndf.head()

,customer_id,date,category
0,50081964,2021-01-25 14:21:07.610,петлі меблеві
2,50081964,2021-01-29 13:53:52.920,лотки для столових приборів
3,50081964,2021-05-31 12:21:22.080,інсектициди
4,50081964,2021-02-20 15:02:06.730,рейлінги для кухні
6,50081964,2021-02-20 15:02:06.730,алюмінієвий профіль


In [29]:
sum(ndf.category.str.isnumeric())

0

In [30]:
categories = ndf.category.unique()

In [31]:
categories

array(['петлі меблеві', 'лотки для столових приборів', 'інсектициди', ...,
       'насоси для фонтанів', 'композитний профіль', 'дитячі шезлонги'],
      dtype=object)

In [32]:
for category in categories:
    processed_category = [word for word in word_tokenize(category)
                              if (word not in stopwords.words("ukrainian")) and word.isalpha()]
    ndf.loc[ndf["category"] == category, "category"] = ' '.join(processed_category)

In [33]:
ndf.head()

,customer_id,date,category
0,50081964,2021-01-25 14:21:07.610,петлі меблеві
2,50081964,2021-01-29 13:53:52.920,лотки столових приборів
3,50081964,2021-05-31 12:21:22.080,інсектициди
4,50081964,2021-02-20 15:02:06.730,рейлінги кухні
6,50081964,2021-02-20 15:02:06.730,алюмінієвий профіль


In [34]:
categories

array(['петлі меблеві', 'лотки для столових приборів', 'інсектициди', ...,
       'насоси для фонтанів', 'композитний профіль', 'дитячі шезлонги'],
      dtype=object)

In [35]:
categories = pd.Series(categories)
categories

0                                          петлі меблеві
1                            лотки для столових приборів
2                                            інсектициди
3                                     рейлінги для кухні
4                                    алюмінієвий профіль
                              ...                       
2124                                  домашні кінотеатри
2125    зарядні пристрої та блоки живлення для ноутбуків
2126                                 насоси для фонтанів
2127                                 композитний профіль
2128                                     дитячі шезлонги
Length: 2129, dtype: object

In [36]:
categories.shape

(2129,)

In [37]:
lemma_categories = nlu.load('uk.lemma').predict(categories, output_level='document')

lemma download started this may take some time.
Approximate size to download 2.3 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


In [38]:
lemma_categories.rename({"document": "category"}, axis=1, inplace=True)

In [39]:
lemma_categories.head()

,category,lem
0,петлі меблеві,"[петля, меблеві]"
1,лотки для столових приборів,"[лотки, для, столовий, приборів]"
2,інсектициди,[інсектициди]
3,рейлінги для кухні,"[рейлінги, для, кухня]"
4,алюмінієвий профіль,"[алюмінієвий, профіль]"


In [40]:
ndf.head()

,customer_id,date,category
0,50081964,2021-01-25 14:21:07.610,петлі меблеві
2,50081964,2021-01-29 13:53:52.920,лотки столових приборів
3,50081964,2021-05-31 12:21:22.080,інсектициди
4,50081964,2021-02-20 15:02:06.730,рейлінги кухні
6,50081964,2021-02-20 15:02:06.730,алюмінієвий профіль


In [41]:
ndf = ndf.merge(lemma_categories, on="category")

In [42]:
ndf

,customer_id,date,category,lem
0,50081964,2021-01-25 14:21:07.610,петлі меблеві,"[петля, меблеві]"
1,50171691,2021-02-17 13:51:00.350,петлі меблеві,"[петля, меблеві]"
2,1600035308,2021-05-09 14:28:16.517,петлі меблеві,"[петля, меблеві]"
3,2000001033,2020-10-17 14:38:24.470,петлі меблеві,"[петля, меблеві]"
4,2000001042,2021-04-20 19:50:38.070,петлі меблеві,"[петля, меблеві]"
...,...,...,...,...
337538,33000053565,2020-10-31 10:45:18.427,спортивні лави,"[спортивний, лава]"
337539,33000056666,2021-09-15 09:38:11.600,ковані хвіртки,"[кований, хвіртка]"
337540,33000074608,2021-05-09 13:18:15.007,домашні кінотеатри,"[домашній, кінотеатр]"
337541,45002082643,2021-06-29 17:45:13.720,композитний профіль,"[композитний, профіль]"


In [58]:
customers_queries = ndf.set_index(["customer_id", "date"]).drop("category", axis=1)

In [59]:
customers_queries.head()["lem"].tolist()

[['петля', 'меблеві'],
 ['петля', 'меблеві'],
 ['петля', 'меблеві'],
 ['петля', 'меблеві'],
 ['петля', 'меблеві']]

In [68]:
customers_vs_queries = pd.DataFrame(customers_queries["lem"].tolist(), index=ndf.customer_id).stack().reset_index().pivot_table(index="customer_id", columns=0, fill_value=0, aggfunc='size')

In [69]:
customers_vs_queries.head()

,bis,iгрові,lego,аварійний,авто,автоакустика,автодиагностики,автозапчастини,автокрісла,автомагнітоли,автомат,автоматика,автоматичний,автомобіль,автомобільний,автопилососи,автосигналізації,автотермометри,автотехнікс,автофарба,автохолодильники,автошампуні,автошини,агроволокно,адміністративний,аератори,аерозольні,акварельні,аквашузи,акрилові,аксесуари,аксесуарів,активний,акумулятори,акумуляторні,алмазна,алмазного,алмазні,алюмінієвий,анкери,...,шпаклівка,шпалери,шпателі,шпатлівка,штангенциркулі,штани,штемпельна,штора,штроборізи,штукатурка,штукатурні,штучний,шумовіброізоляція,шумівки,шурупи,шуруповерти,щебінь,щит,щоденник,щітки,ягода,яйцерізки,ємність,іграшка,ігровий,ізоляційні,ізоляція,інвентар,інвертори,індексні,індивідуальний,індикатор,іноземний,інсектициди,інсталяції,інструмент,інтерактивні,інформаційний,інфрачервоні,ґрунтовка
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
50081964,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
50095759,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
50097247,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
50113709,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
50116694,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [74]:
customers_vs_queries[customers_vs_queries > 0] = 1

In [76]:
customers_vs_queries.describe()

,bis,iгрові,lego,аварійний,авто,автоакустика,автодиагностики,автозапчастини,автокрісла,автомагнітоли,автомат,автоматика,автоматичний,автомобіль,автомобільний,автопилососи,автосигналізації,автотермометри,автотехнікс,автофарба,автохолодильники,автошампуні,автошини,агроволокно,адміністративний,аератори,аерозольні,акварельні,аквашузи,акрилові,аксесуари,аксесуарів,активний,акумулятори,акумуляторні,алмазна,алмазного,алмазні,алюмінієвий,анкери,...,шпаклівка,шпалери,шпателі,шпатлівка,штангенциркулі,штани,штемпельна,штора,штроборізи,штукатурка,штукатурні,штучний,шумовіброізоляція,шумівки,шурупи,шуруповерти,щебінь,щит,щоденник,щітки,ягода,яйцерізки,ємність,іграшка,ігровий,ізоляційні,ізоляція,інвентар,інвертори,індексні,індивідуальний,індикатор,іноземний,інсектициди,інсталяції,інструмент,інтерактивні,інформаційний,інфрачервоні,ґрунтовка
count,"20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00",...,"20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00"
mean,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.02,0.22,0.00,0.00,0.00,0.00,0.03,0.00,0.03,0.01,0.03,0.03,0.00,0.08,0.00,0.00,0.01,0.02,0.00,0.02,0.02,0.01,0.00,0.00,0.03,0.02,0.05,...,0.08,0.12,0.11,0.01,0.00,0.00,0.00,0.08,0.00,0.05,0.03,0.03,0.02,0.01,0.14,0.01,0.00,0.00,0.03,0.05,0.00,0.00,0.06,0.02,0.08,0.09,0.02,0.01,0.00,0.01,0.00,0.00,0.00,0.03,0.00,0.03,0.01,0.00,0.00,0.11
std,0.04,0.01,0.12,0.11,0.06,0.02,0.01,0.01,0.06,0.01,0.04,0.03,0.18,0.15,0.42,0.04,0.01,0.02,0.01,0.17,0.02,0.16,0.12,0.17,0.17,0.03,0.27,0.07,0.02,0.08,0.16,0.01,0.12,0.15,0.07,0.03,0.01,0.18,0.15,0.21,...,0.27,0.32,0.32,0.09,0.06,0.04,0.04,0.28,0.01,0.21,0.17,0.17,0.15,0.09,0.34,0.12,0.06,0.06,0.17,0.23,0.06,0.06,0.24,0.13,0.27,0.29,0.15,0.08,0.05,0.09,0.02,0.02,0.01,0.16,0.05,0.17,0.09,0.01,0.06,0.32
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
75%,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.